In [3]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import numpy as np
import pandas.io.sql as pd_sql
import pickle

# We are also going to do some basic viz
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Postgres info to connect

connection_args = {
    'host': '54.153.60.68', # You'll have to update this to your IP
    'user': 'ubuntu',    # username
    'dbname': 'tennis',   # DB that we are connecting to
    'port': 5432         # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)
cursor = connection.cursor()

In [13]:
query = """
CREATE TABLE  wim_ladies_2013 (
      player1 VARCHAR(255),
      player2 VARCHAR(255),
      round INT,
      result INT,
      fnl1 DOUBLE PRECISION,
      fnl2 DOUBLE PRECISION,
      fsp_1 DOUBLE PRECISION,
      fsw_1 DOUBLE PRECISION,
      ssp_1 DOUBLE PRECISION,
      ssw_1 DOUBLE PRECISION,
      ace_1 INT,
      dbf_1 INT,
      wnr_1 INT,
      ufe_1 INT,
      bpc_1 INT,
      bpw_1 INT,
      npa_1 INT,
      npw_1 INT,
      tpw_1 INT,
      st1_1 INT,
      st2_1 INT,
      st3_1 INT,
      st4_1 INT,
      st5_1 INT,
      fsp_2 DOUBLE PRECISION,
      fsw_2 DOUBLE PRECISION,
      ssp_2 DOUBLE PRECISION,
      ssw_2 DOUBLE PRECISION,
      ace_2 INT,
      dbf_2 INT,
      wnr_2 INT,
      ufe_2 INT,
      bpc_2 INT,
      bpw_2 INT,
      npa_2 INT,
      npw_2 INT,
      tpw_2 INT,
      st1_2 INT,
      st2_2 INT,
      st3_2 INT,
      st4_2 INT,
      st5_2 INT);
"""
cursor.execute(query)
cursor.execute('commit')

In [30]:
def get_table_head(cursor, table):
    # Check column data types

    colnames = get_table_cols(cursor, table)
    print (colnames)
    
    query = """
    SELECT * FROM {} LIMIT 5;
    """.format(table)
    
    cursor.execute(query)
    content = cursor.fetchall()
    print(content)
    
    table_first5 = pd.DataFrame(content, columns=colnames)

    return table_first5

In [23]:
def get_table_cols(cursor, table):
    # Check column data types
    query = """
    SELECT column_name, data_type 
    FROM information_schema.columns
    WHERE table_name = '{}'
    """.format(table)

    cursor.execute(query)
    return [col[0] for col in cursor.fetchall()]

In [31]:
get_table_head(cursor, 'wim_ladies_2013')

['player1', 'player2', 'round', 'result', 'fnl1', 'fnl2', 'fsp_1', 'fsw_1', 'ssp_1', 'ssw_1', 'ace_1', 'dbf_1', 'wnr_1', 'ufe_1', 'bpc_1', 'bpw_1', 'npa_1', 'npw_1', 'tpw_1', 'st1_1', 'st2_1', 'st3_1', 'st4_1', 'st5_1', 'fsp_2', 'fsw_2', 'ssp_2', 'ssw_2', 'ace_2', 'dbf_2', 'wnr_2', 'ufe_2', 'bpc_2', 'bpw_2', 'npa_2', 'npw_2', 'tpw_2', 'st1_2', 'st2_2', 'st3_2', 'st4_2', 'st5_2']
[('M.Koehler', 'V.Azarenka', 1, 0, 0.0, 2.0, 60.0, 21.0, 40.0, 8.0, 2, 4, 19, 18, 7, 2, 3, 3, None, 1, 2, None, None, None, 72.0, 31.0, 28.0, 4.0, 3, 9, 25, 14, 16, 6, 8, 4, None, 6, 6, None, None, None), ('E.Baltacha', 'F.Pennetta', 1, 0, 0.0, 2.0, 69.0, 23.0, 31.0, 6.0, 0, 0, 12, 10, 1, 1, 14, 10, None, 4, 1, None, None, None, 53.0, 18.0, 47.0, 15.0, 4, 4, 22, 14, 6, 5, 14, 11, None, 6, 6, None, None, None), ('S-W.Hsieh', 'T.Maria', 1, 1, 2.0, 0.0, 63.0, 17.0, 37.0, 10.0, 1, 1, 23, 13, 14, 6, 13, 8, None, 6, 6, None, None, None, 56.0, 16.0, 44.0, 7.0, 2, 10, 11, 29, 1, 0, 8, 2, None, 1, 0, None, None, None), 

,player1,player2,round,result,fnl1,fnl2,fsp_1,fsw_1,ssp_1,ssw_1,...,bpc_2,bpw_2,npa_2,npw_2,tpw_2,st1_2,st2_2,st3_2,st4_2,st5_2
0,M.Koehler,V.Azarenka,1,0,0.0,2.0,60.0,21.0,40.0,8.0,...,16,6,8,4,None,6,6,NaN,None,None
1,E.Baltacha,F.Pennetta,1,0,0.0,2.0,69.0,23.0,31.0,6.0,...,6,5,14,11,None,6,6,NaN,None,None
2,S-W.Hsieh,T.Maria,1,1,2.0,0.0,63.0,17.0,37.0,10.0,...,1,0,8,2,None,1,0,NaN,None,None
3,A.Cornet,V.King,1,1,2.0,1.0,57.0,36.0,43.0,21.0,...,4,1,48,32,None,6,3,1.0,None,None
4,Y.Putintseva,K.Flipkens,1,0,0.0,2.0,73.0,34.0,27.0,12.0,...,9,3,35,24,None,7,6,NaN,None,None


In [35]:
def get_table_all(cursor, table):
    # Check column data types

    colnames = get_table_cols(cursor, table)
    
    query = """
    SELECT * FROM {};
    """.format(table)
    
    cursor.execute(query)
    content = cursor.fetchall()
    
    table_first5 = pd.DataFrame(content, columns=colnames)

    return table_first5

In [36]:
wim_ladies = get_table_all(cursor, 'wim_ladies_2013')

In [43]:
query = '''
SELECT
    player1, player2, result
FROM
    us_men_2013;
'''

cursor.execute(query)
cursor.fetchall()

[('Richard Gasquet', 'Michael Russell', 1),
 ('Stephane Robert', 'Albano Olivetti', 1),
 ('Jan-Lennard Struff', 'Guillaume Rufin', 0),
 ('Aljaz Bedene', 'Dmitry Tursunov', 0),
 ('Feliciano Lopez', 'Florent Serra', 1),
 ('Kenny De Schepper', 'Bradley Klahn', 0),
 ('Andrey Kuznetsov', 'Dudi Sela', 0),
 ('Pablo Cuevas', 'Janko Tipsarevic', 0),
 ('Ernests Gulbis', 'Andreas Haider-Maurer', 0),
 ('Mikhail Kukushkin', 'Andrej Martin', 1),
 ('Roberto Bautista Agut', 'Thomaz Bellucci', 1),
 ('Nick Kyrgios', 'David Ferrer', 0),
 ('Tommy Robredo', 'Marinko Matosevic', 1),
 ('Robin Haase', 'Frank Dancevic', 0),
 ('Albert Ramos', 'Bernard Tomic', 0),
 ('Daniel Evans', 'Kei Nishikori', 1),
 ('Fernando Verdasco', 'Ivan Dodig', 0),
 ('Rhyne Williams', 'Nikolay Davydenko', 0),
 ('Ryan Harrison', 'Rafael Nadal', 0),
 ('Novak Djokovic', 'Ricardas Berankis', 1),
 ('Benjamin Becker', 'Lukas Rosol', 1),
 ('Lukasz Kubot', 'Jarkko Nieminen', 0),
 ('Joao Sousa', 'Grigor Dimitrov', 1),
 ('Tommy Haas', 'Paul-Hen

In [44]:
query = '''
SELECT
    player1, player2, result
FROM
    us_men_2013
WHERE
    player1 = 'Richard Gasquet';
'''

cursor.execute(query)
cursor.fetchall()

[('Richard Gasquet', 'Michael Russell', 1),
 ('Richard Gasquet', 'Stephane Robert', 1),
 ('Richard Gasquet', 'Dmitry Tursunov', 1),
 ('Richard Gasquet', 'Milos Raonic', 1),
 ('Richard Gasquet', 'David Ferrer', 1),
 ('Richard Gasquet', 'Rafael Nadal', 0)]

In [46]:
query = '''
SELECT
    player1, player2, result
FROM
    us_men_2013
WHERE
    player1 = 'Richard Gasquet' 
OR 
    player2 = 'Richard Gasquet';
'''

cursor.execute(query)
cursor.fetchall()

[('Richard Gasquet', 'Michael Russell', 1),
 ('Richard Gasquet', 'Stephane Robert', 1),
 ('Richard Gasquet', 'Dmitry Tursunov', 1),
 ('Richard Gasquet', 'Milos Raonic', 1),
 ('Richard Gasquet', 'David Ferrer', 1),
 ('Richard Gasquet', 'Rafael Nadal', 0)]

In [47]:
query = '''
SELECT COUNT(*)
FROM us_men_2013;
'''

cursor.execute(query)
cursor.fetchall()

[(126,)]

In [48]:
query = '''
SELECT player1, COUNT(*)
FROM us_men_2013
GROUP BY player1;
'''

cursor.execute(query)
cursor.fetchall()

[('Jiri Vesely', 1),
 ('Tommy Haas', 3),
 ('Sergiy Stakhovsky', 1),
 ('Paolo Lorenzi', 1),
 ('Jurgen Zopp', 1),
 ('Ernests Gulbis', 1),
 ('Collin Altamirano', 1),
 ('Rajeev Ram', 1),
 ('Jarkko Nieminen', 1),
 ('John Isner', 3),
 ('Aljaz Bedene', 1),
 ('Richard Gasquet', 6),
 ('Jan-Lennard Struff', 1),
 ('Denis Kudla', 1),
 ('Igor Sijsling', 1),
 ('Lukasz Kubot', 1),
 ('Lukas Lacko', 1),
 ('Tobias Kamke', 2),
 ('Pablo Andujar', 2),
 ('Guillermo Garcia-Lopez', 1),
 ('James Blake', 1),
 ('Albert Ramos', 1),
 ('Dudi Sela', 1),
 ('Roger Federer', 4),
 ('Kenny De Schepper', 1),
 ('Yen-Hsun Lu', 1),
 ('Donald Young', 2),
 ('Adrian Mannarino', 1),
 ('Andrey Kuznetsov', 1),
 ('Santiago Giraldo', 1),
 ('Marcos Baghdatis', 2),
 ('Andreas Haider-Maurer', 1),
 ('Robin Haase', 1),
 ('Fernando Verdasco', 1),
 ('Florian Mayer', 1),
 ('Benoit Paire', 1),
 ('Evgeny Donskoy', 2),
 ('Stephane Robert', 1),
 ('Edouard Roger-Vasselin', 1),
 ('Roberto Bautista Agut', 2),
 ('Thomas Fabbiano', 1),
 ('Jurgen Mel

In [57]:
query = '''
SELECT player1, AVG(result)
FROM us_men_2013
GROUP BY player1;
'''

cursor.execute(query)
cursor.fetchall()

[('Jiri Vesely', Decimal('0E-20')),
 ('Tommy Haas', Decimal('0.66666666666666666667')),
 ('Sergiy Stakhovsky', Decimal('0E-20')),
 ('Paolo Lorenzi', Decimal('0E-20')),
 ('Jurgen Zopp', Decimal('0E-20')),
 ('Ernests Gulbis', Decimal('0E-20')),
 ('Collin Altamirano', Decimal('0E-20')),
 ('Rajeev Ram', Decimal('1.00000000000000000000')),
 ('Jarkko Nieminen', Decimal('0E-20')),
 ('John Isner', Decimal('0.66666666666666666667')),
 ('Aljaz Bedene', Decimal('0E-20')),
 ('Richard Gasquet', Decimal('0.83333333333333333333')),
 ('Jan-Lennard Struff', Decimal('0E-20')),
 ('Denis Kudla', Decimal('0E-20')),
 ('Igor Sijsling', Decimal('0E-20')),
 ('Lukasz Kubot', Decimal('0E-20')),
 ('Lukas Lacko', Decimal('0E-20')),
 ('Tobias Kamke', Decimal('0.50000000000000000000')),
 ('Pablo Andujar', Decimal('0.50000000000000000000')),
 ('Guillermo Garcia-Lopez', Decimal('0E-20')),
 ('James Blake', Decimal('0E-20')),
 ('Albert Ramos', Decimal('0E-20')),
 ('Dudi Sela', Decimal('0E-20')),
 ('Roger Federer', Decim

In [58]:
query = '''
SELECT COUNT(*)
FROM us_men_2013
WHERE result = 1;
'''

cursor.execute(query)
cursor.fetchall()

[(59,)]

In [61]:
query = '''
SELECT 
      us_men_2013.player1, us_men_2013.tpw_1, fr_men_2013.tpw_1 
FROM 
      us_men_2013, fr_men_2013 
WHERE 
      us_men_2013.player1 = fr_men_2013.player1;

'''

cursor.execute(query)
cursor.fetchall()

[('Jan-Lennard Struff', 149, 132),
 ('Feliciano Lopez', 148, 80),
 ('Feliciano Lopez', 148, 126),
 ('Kenny De Schepper', 133, 125),
 ('Andrey Kuznetsov', 183, 94),
 ('Ernests Gulbis', 159, 97),
 ('Roberto Bautista Agut', 87, 80),
 ('Roberto Bautista Agut', 87, 100),
 ('Nick Kyrgios', 75, 57),
 ('Tommy Robredo', 138, 46),
 ('Tommy Robredo', 138, 169),
 ('Robin Haase', 149, 115),
 ('Albert Ramos', 165, 94),
 ('Fernando Verdasco', 144, 187),
 ('Fernando Verdasco', 144, 89),
 ('Ryan Harrison', 69, 160),
 ('Novak Djokovic', 91, 158),
 ('Novak Djokovic', 91, 116),
 ('Novak Djokovic', 91, 134),
 ('Novak Djokovic', 91, 94),
 ('Novak Djokovic', 91, 93),
 ('Novak Djokovic', 91, 114),
 ('Benjamin Becker', 125, 85),
 ('Lukasz Kubot', 86, 90),
 ('Joao Sousa', 145, 87),
 ('Tommy Haas', 89, 82),
 ('Tommy Haas', 89, 224),
 ('Tommy Haas', 89, 116),
 ('Tommy Haas', 89, 101),
 ('Florian Mayer', 77, 87),
 ('Tobias Kamke', 157, 140),
 ('Denis Istomin', 112, 80),
 ('Julien Benneteau', 125, 64),
 ('Julien Be

In [67]:
query = '''
SELECT
      us_men_2013.player1, 
      SUM(us_men_2013.tpw_1) AS us_points, 
      SUM(french_men_2013.tpw_1) AS french_points, 
      SUM(us_men_2013.tpw_1 + french_men_2013.tpw_1) AS total_points 
FROM
      us_men_2013, french_men_2013 
WHERE 
      us_men_2013.player1 = french_men_2013.player1
GROUP BY
      us_men_2013.player1;
'''

cursor.execute(query)
cursor.fetchall()

[('Jiri Vesely', 112, 120, 232),
 ('Tommy Haas', 1208, 1569, 2777),
 ('Sergiy Stakhovsky', 156, 57, 213),
 ('Joao Sousa', 145, 87, 232),
 ('Jurgen Zopp', 116, 58, 174),
 ('Feliciano Lopez', 804, 618, 1422),
 ('Benjamin Becker', 125, 85, 210),
 ('Denis Istomin', 112, 80, 192),
 ('Stanislas Wawrinka', 1968, 2336, 4304),
 ('Alexandr Dolgopolov', 84, 116, 200),
 ('Ernests Gulbis', 159, 97, 256),
 ('Novak Djokovic', 4014, 4254, 8268),
 ('Jarkko Nieminen', 138, 172, 310),
 ('Jan-Lennard Struff', 149, 132, 281),
 ('Igor Sijsling', 214, 226, 440),
 ('Lukasz Kubot', 86, 90, 176),
 ('Lukas Lacko', 133, 72, 205),
 ('Tobias Kamke', 219, 280, 499),
 ('Albert Ramos', 165, 94, 259),
 ('Kenny De Schepper', 133, 125, 258),
 ('Adrian Mannarino', 157, 160, 317),
 ('Andrey Kuznetsov', 183, 94, 277),
 ('Andreas Haider-Maurer', 91, 99, 190),
 ('Robin Haase', 149, 115, 264),
 ('Fernando Verdasco', 288, 276, 564),
 ('Florian Mayer', 77, 87, 164),
 ('Benoit Paire', 573, 322, 895),
 ('Evgeny Donskoy', 266, 240,

### Challenge 1: find the number of matches played by each player in each tournament. 

In [77]:
query = '''
select * from information_schema.tables
where table_schema = 'public';
'''

cursor.execute(query)
result = cursor.fetchall()

In [83]:
tables = [table[2] for table in result]

In [124]:
result = []
for table in tables:
    query = '''
    SELECT player, SUM(totalmatches) AS totalmaches
    FROM
        (
        SELECT player1 AS player, CAST (COUNT(*) AS INT) AS totalmatches
        FROM {}
        GROUP BY player1
        UNION ALL
        SELECT player2 AS player, CAST (COUNT(*) AS INT) AS totalmatches
        FROM {}
        GROUP BY player2
        ) s
    GROUP BY
    s.player;
    '''.format(table, table)
    cursor.execute(query)
    count = cursor.fetchall()
    count_match = [x + (table,) for x in count]
    result.extend(count_match)
    #print (count[0])
    #result.extend({}.join(count).format(table))
    #print ('Number of matches in {}: \n'.format(table), count)

In [125]:
match_count = pd.DataFrame(result, columns=['Player','Match Count', 'Country'])

In [127]:
match_count

,Player,Match Count,Country
0,Angelique Kerber,4,aus_ladies_2013
1,Kiki Bertens,1,aus_ladies_2013
2,Sorana Cirstea,1,aus_ladies_2013
3,Petra Kvitova,1,aus_ladies_2013
4,Anna Schmiedlova,2,aus_ladies_2013
5,Tsvetana Pironkova,2,aus_ladies_2013
6,Yvonne Meusburger,3,aus_ladies_2013
7,Luksika Kumkhum,2,aus_ladies_2013
8,Barbora Zahlavova Strycova,2,aus_ladies_2013
9,Venus Williams,1,aus_ladies_2013


### Challenge 2: Who has played the most matches total in all games 

In [129]:
tables

['aus_ladies_2013',
 'aus_men_2013',
 'french_men_2013',
 'french_ladies_2013',
 'us_ladies_2013',
 'us_men_2013',
 'wimbledon_men_2013',
 'wimbledon_ladies_2013']

In [ ]:

query = '''
CREATE VIEW combined_matches AS 
(
SELECT * 
FROM french_ladies_2013
CROSS JOIN us_ladies_2013
CROSS JOIN aus_ladies_2013
);
'''

cursor.execute(query)
result = cursor.fetchall()

In [134]:
result

[('Su-Wei Hsieh',
  'Maria Sharapova',
  1,
  0,
  0.0,
  2.0,
  62.0,
  18.0,
  38.0,
  5.0,
  0,
  3,
  8,
  8,
  0,
  0,
  1,
  3,
  31,
  2,
  1,
  None,
  None,
  None,
  68.0,
  25.0,
  32.0,
  8.0,
  4,
  0,
  25,
  19,
  4,
  6,
  3,
  5,
  57,
  6,
  6,
  None,
  None,
  None,
  'S Williams',
  'V Azarenka',
  7,
  1,
  2.0,
  1.0,
  57.0,
  44.0,
  43.0,
  20.0,
  9,
  5,
  36,
  35,
  12,
  7,
  13,
  4,
  None,
  7,
  6,
  6,
  None,
  None,
  57.0,
  31.0,
  43.0,
  23.0,
  2,
  7,
  17,
  27,
  8,
  4,
  15,
  10,
  None,
  5,
  7,
  1,
  None,
  None),
 ('Su-Wei Hsieh',
  'Maria Sharapova',
  1,
  0,
  0.0,
  2.0,
  62.0,
  18.0,
  38.0,
  5.0,
  0,
  3,
  8,
  8,
  0,
  0,
  1,
  3,
  31,
  2,
  1,
  None,
  None,
  None,
  68.0,
  25.0,
  32.0,
  8.0,
  4,
  0,
  25,
  19,
  4,
  6,
  3,
  5,
  57,
  6,
  6,
  None,
  None,
  None,
  'F Pennetta',
  'V Azarenka',
  6,
  0,
  0.0,
  2.0,
  44.0,
  12.0,
  56.0,
  7.0,
  1,
  3,
  19,
  23,
  6,
  5,
  20,
  12,
  None,
